## УАВАВЫАЫАЫА

In [1]:
!pip install transformers

In [2]:
!pip install ipywidgets widgetsnbextension pandas-profiling
!jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 36.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.1
    Uninstalling scipy-1.11.1:
      Successfully uninstalled scipy-1.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.6.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.6.0 requires dask==2023.3.2, but you have dask 2023.7.0 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.10.1 which is incompatible.
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - V

In [3]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split

import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm_notebook as tqdm

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/s

In [4]:
all_files = glob.glob('/kaggle/input/datasets-cp' + "/dataset_*.csv")

# Читайте каждый файл и сохраняйте его содержимое в список датафреймов
list_of_datasets = [pd.read_csv(file) for file in all_files]

# Объедините все датафреймы в один
combined_dataframe = pd.concat(list_of_datasets, ignore_index=True)

In [5]:
combined_dataframe.drop('Unnamed: 0', axis=1, inplace=True)
combined_dataframe = combined_dataframe.dropna()

In [6]:
combined_dataframe.head()

,address,target_building_id,target_address
0,"196641, Санкт-Петербург г, Школьная ул, д.5",76337.0,"Санкт-Петербург, город Павловск, Пязелево, Шко..."
1,"Санкт-Петербург (г.), Железнодорожный (пр-кт.)...",220011.0,"г.Санкт-Петербург, Железнодорожный проспект, д..."
2,"198325, Санкт-Петербург г, Красное Село г, Теа...",109979.0,"Санкт-Петербург, город Красное Село, Дудергоф,..."
3,"195256, Санкт-Петербург (г.), Верности (ул.), ...",72112.0,"г.Санкт-Петербург, улица Верности, дом 28, кор..."
4,"194291, Санкт-Петербург (г.), Кустодиева (ул.)...",186531.0,"г.Санкт-Петербург, улица Кустодиева, дом 1, ли..."


In [7]:
def process_address(address: str):
    short_to_full = {'бульв': 'бульвар', 'дор': 'дорога',
                     'наб': 'набережная', 'пер': 'переулок', 'пл': 'площадь', 'пр': 'проспект',
                     'ул': 'улица', 'д': 'дом', 'к': 'корпус', 'г': 'город', 'б-р': 'бульвар',
                     'пр-кт': 'проспект'}
    punctuation = [("(", ""), (")", ""), (".", " "), (',', ' ,')]
    for (old, new) in punctuation:
        address = address.replace(old, new)
    address = address.split()
    if address[0] == "россия":
        address[0] = ""
    for index, word in enumerate(address):
        if word.isdigit() and len(word) == 6:
            address[index] = ""
        else:
            address[index] = short_to_full.get(word, word)
    while address[0] == "," or address[0] == "":
        address = address[1:]
    return " ".join(address).replace(' ,', ',')

In [8]:
combined_dataframe['address'] = combined_dataframe.address.apply(process_address)

In [9]:
combined_dataframe.head()

,address,target_building_id,target_address
0,"Санкт-Петербург город, Школьная улица, дом 5",76337.0,"Санкт-Петербург, город Павловск, Пязелево, Шко..."
1,"Санкт-Петербург город, Железнодорожный проспек...",220011.0,"г.Санкт-Петербург, Железнодорожный проспект, д..."
2,"Санкт-Петербург город, Красное Село город, Теа...",109979.0,"Санкт-Петербург, город Красное Село, Дудергоф,..."
3,"Санкт-Петербург город, Верности улица, дом 28,...",72112.0,"г.Санкт-Петербург, улица Верности, дом 28, кор..."
4,"Санкт-Петербург город, Кустодиева улица, дом 1...",186531.0,"г.Санкт-Петербург, улица Кустодиева, дом 1, ли..."


In [10]:
#combined_dataframe = pd.read_csv("/kaggle/input/datasets-cp/dataset_1.csv")
#combined_dataframe.drop('Unnamed: 0', axis=1, inplace=True)
#combined_dataframe.dropna(inplace=True)

In [11]:
combined_dataframe = combined_dataframe.iloc[0:20000] # for test

In [12]:
import torch
from sklearn.model_selection import train_test_split
from transformers import BartTokenizer, BartForConditionalGeneration, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

In [13]:
class AddressCorrectionDataset(Dataset):
    def __init__(self, tokenizer, dataframe, max_length=128):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        src_text = self.dataframe.iloc[idx, 0]
        tgt_text = self.dataframe.iloc[idx, 1]
        src_encoding = self.tokenizer(src_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        tgt_encoding = self.tokenizer(tgt_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        return {
            "src_input_ids": src_encoding["input_ids"].squeeze(),
            "src_attention_mask": src_encoding["attention_mask"].squeeze(),
            "tgt_input_ids": tgt_encoding["input_ids"].squeeze(),
            "tgt_attention_mask": tgt_encoding["attention_mask"].squeeze()
        }

In [14]:
combined_dataframe = combined_dataframe[['address', 'target_address']]
train_df, test_df = train_test_split(combined_dataframe, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

train_dataset = AddressCorrectionDataset(tokenizer, train_df)
val_dataset = AddressCorrectionDataset(tokenizer, val_df)
test_dataset = AddressCorrectionDataset(tokenizer, test_df)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large").to(device) #bart-large
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3) # 3 эпохи

for epoch in range(3):
    model.train()
    train_loss = 0
    # добавьте tqdm в цикл
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training", total=len(train_loader)):
        optimizer.zero_grad()
        input_ids = batch["src_input_ids"].to(device)
        attention_mask = batch["src_attention_mask"].to(device)
        labels = batch["tgt_input_ids"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}")
    
    model.eval()
    val_loss = 0
    # добавьте tqdm и для валидационного цикла
    for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} Validation", total=len(val_loader)):
        input_ids = batch["src_input_ids"].to(device)
        attention_mask = batch["src_attention_mask"].to(device)
        labels = batch["tgt_input_ids"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss}")

Epoch 1 Training:   0%|          | 0/2025 [00:00<?, ?it/s]

Epoch 1, Training Loss: 0.23496473402474766


Epoch 1 Validation:   0%|          | 0/225 [00:00<?, ?it/s]

Epoch 1, Validation Loss: 0.053325116237004595


Epoch 2 Training:   0%|          | 0/2025 [00:00<?, ?it/s]

Epoch 2, Training Loss: 0.03948135677426134


Epoch 2 Validation:   0%|          | 0/225 [00:00<?, ?it/s]

Epoch 2, Validation Loss: 0.04266950005458461


Epoch 3 Training:   0%|          | 0/2025 [00:00<?, ?it/s]

In [ ]:
for addr in combined_dataframe.address.head(20).tolist():
    print(addr)
    print((process_address(addr)))
    print('----')

In [26]:
pd.set_option('display.max_colwidth', None)

In [79]:
combined_dataframe.head()

,address,target_building_id
0,"Санкт-Петербург город, Школьная улица, дом 5",76337.0
1,"Санкт-Петербург город, Железнодорожный проспект, дом 14 2 1",220011.0
2,"Санкт-Петербург город, Красное Село город, Театральная Дудергоф улица, дом 9",109979.0
3,"Санкт-Петербург город, Верности улица, дом 28, Строение 2",72112.0
4,"Санкт-Петербург город, Кустодиева улица, дом 1, Строение А",186531.0


## target ID ( плохо работает )

In [118]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [16]:
combined_dataframe = combined_dataframe.iloc[0:5000]

In [17]:
from sklearn.preprocessing import LabelEncoder

# Создаем экземпляр LabelEncoder
label_encoder = LabelEncoder()

# Обучаем encoder и преобразуем target_building_id в последовательные метки
combined_dataframe['encoded_building_id'] = label_encoder.fit_transform(combined_dataframe['target_building_id'])

# Сохраняем отображение между исходными ID и закодированными метками (это необязательный шаг, но он может быть полезен)
id_to_encoded = dict(zip(combined_dataframe['target_building_id'], combined_dataframe['encoded_building_id']))
encoded_to_id = {v: k for k, v in id_to_encoded.items()}

# Теперь используйте 'encoded_building_id' вместо 'target_building_id' при создании набора данных и обучении модели


In [18]:
n_classes = combined_dataframe["encoded_building_id"].nunique()

In [19]:
assert combined_dataframe['encoded_building_id'].min() == 0
assert combined_dataframe['encoded_building_id'].max() == n_classes - 1

In [24]:
combined_dataframe.encoded_building_id.max()

4751

In [21]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Создаем датасет для загрузки данных
class AddressClassificationDataset(Dataset):
    def __init__(self, tokenizer, dataframe, max_length=128):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Загрузим и подготовим наши данные
combined_dataframe = combined_dataframe[['address', 'encoded_building_id']]
train_df, test_df = train_test_split(combined_dataframe, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
n_classes = combined_dataframe["encoded_building_id"].nunique()

train_dataset = AddressClassificationDataset(tokenizer, train_df)
val_dataset = AddressClassificationDataset(tokenizer, val_df)
test_dataset = AddressClassificationDataset(tokenizer, test_df)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=n_classes).to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)

# Тренировка модели
for epoch in range(3):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training", total=len(train_loader)):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}")
    
    model.eval()
    val_loss = 0
    for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} Validation", total=len(val_loader)):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss}")

# Сохраните вашу модель после обучения для дальнейшего использования или развертывания
model.save_pretrained("./trained_rubert_classifier/")
tokenizer.save_pretrained("./trained_rubert_classifier/")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 Training: 100%|██████████| 507/507 [05:33<00:00,  1.52it/s]


Epoch 1, Training Loss: 8.569904329273593


Epoch 1 Validation: 100%|██████████| 57/57 [00:09<00:00,  6.04it/s]


Epoch 1, Validation Loss: 9.24963665008545


Epoch 2 Training: 100%|██████████| 507/507 [05:36<00:00,  1.50it/s]


...

Epoch 3 Training: 100%|██████████| 507/507 [05:40<00:00,  1.49it/s]


Epoch 3, Training Loss: 8.34999401047385


Epoch 3 Validation: 100%|██████████| 57/57 [00:09<00:00,  6.10it/s]


Epoch 3, Validation Loss: 9.796061850430673


('./trained_rubert_classifier/tokenizer_config.json',
 './trained_rubert_classifier/special_tokens_map.json',
 './trained_rubert_classifier/vocab.txt',
 './trained_rubert_classifier/added_tokens.json')

In [34]:
def predict_building_id(text):
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**encoding).logits
    predicted_label_idx = torch.argmax(logits, dim=1).item()
    return encoded_to_id[predicted_label_idx]

# Пример проверки
test_address = "Санкт-Петербург 1, Строение Аfdfsddfsf"  # Замените на ваш пример некорректного адреса
predicted_id = predict_building_id(test_address)
print(f"Predicted building ID for address '{test_address}': {predicted_id}")



Predicted building ID for address 'Санкт-Петербург 1, Строение Аfdfsddfsf': 88914.0


In [27]:
combined_dataframe.head()

,address,encoded_building_id
0,"196641, Санкт-Петербург г, Школьная ул, д.5",2818
1,"Санкт-Петербург (г.), Железнодорожный (пр-кт.), д.14 2 1",4720
2,"198325, Санкт-Петербург г, Красное Село г, Театральная (Дудергоф) ул, д.9",4008
3,"195256, Санкт-Петербург (г.), Верности (ул.), д.28, Строение 2",2668
4,"194291, Санкт-Петербург (г.), Кустодиева (ул.), д.1, Строение А",4425
